In [9]:
from datetime import datetime, date

def calculate_age(birth_date):
    today = date.today()
    age = today.year - birth_date.year

    if (today.month, today.day) < (birth_date.month, birth_date.day):
        age -= 1

    return age


def main():
    try:
        user_input = input("Enter your birth date (mm/dd/yyyy): ")
        birth_date = datetime.strptime(user_input, "%m/%d/%Y").date()
        age = calculate_age(birth_date)
        european_format = birth_date.strftime("%d/%m/%Y")

        print("\n Birth date accepted.")
        print(f" Your age is: {age} years")
        print(f" European format: {european_format}")

    except ValueError:
        print("\n Invalid input.")
        print("Please enter a valid date in mm/dd/yyyy format (e.g: 08/25/2001)")

if __name__ == "__main__":
    main()

Enter your birth date (mm/dd/yyyy):  04/24/2000



 Birth date accepted.
 Your age is: 25 years
 European format: 24/04/2000


In [29]:
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

def main():
    try:
        start = int(input("Enter the start range:"))
        end = int(input("Enter the end range:"))
        if start <= 0 or end <= 0 or start > end:
            print("Invalid input! Please enter positive integers with start <= end.")
            return
        
        primes = [num for num in range(start, end + 1) if is_prime(num)]
        
        print(f"Prime numbers in range [{start}, {end}]:")
        for i in range(0, len(primes), 10):
            print(' '.join(map(str, primes[i:i+10])))
        
        if not primes:
            print("No prime numbers found in the given range.")
            
    except ValueError:
        print("Invalid input! Please enter positive integers.")

if __name__ == "__main__":
    main()


Enter the start range: 24
Enter the end range: 98


Prime numbers in range [24, 98]:
29 31 37 41 43 47 53 59 61 67
71 73 79 83 89 97


In [69]:
data = """RegNum,Exam,Coursework
STU001,85,78
STU002,92,88
STU003,95,82
STU004,63,70
STU005,55,55"""
with open('marks.txt', 'w') as f:
    f.write(data)
print("marks.txt created!")

marks.txt created!


In [79]:
import numpy as np
import sys
from collections import Counter

def compute_overall_mark(exam, coursework):
    return round(0.7 * exam + 0.3 * coursework, 2)

def assign_grade(overall):
    if overall >= 70:
        return 'A'
    elif overall >= 60:
        return 'B'
    elif overall >= 50:
        return 'C'
    elif overall >= 40:
        return 'D'
    else:
        return 'F'

def main():
    try:
        input_file = input("Enter input filename(enter marks.txt): ").strip()
        
        students = []
        with open(input_file, 'r') as f:
            next(f)  
            for line_num, line in enumerate(f, 2):
                parts = line.strip().split(',')
                if len(parts) != 3:
                    print(f"Warning: Skipping invalid line {line_num}: {line.strip()}")
                    continue
                reg_num, exam_str, cw_str = parts
                exam = float(exam_str)
                coursework = float(cw_str)
                
                if not (0 <= exam <= 100 and 0 <= coursework <= 100):
                    print(f"Warning: Invalid marks on line {line_num}, skipping")
                    continue
                
                overall = compute_overall_mark(exam, coursework)
                grade = assign_grade(overall)
                students.append((reg_num.strip(), exam, coursework, overall, grade))
        
        if not students:
            print("No valid student data found!")
            return
        
        dtype = [('reg_num', 'U10'), ('exam', float), ('coursework', float), 
                ('overall', float), ('grade', 'U1')]
        student_array = np.array(students, dtype=dtype)
        
        sorted_array = np.sort(student_array, order='overall')[::-1]
        
        grades = [s['grade'] for s in sorted_array]
        stats = Counter(grades)
        print("\n Grade Statistics:")
        for grade in sorted(stats, key=lambda g: 'A B C D F'.find(g)):
            count = stats[grade]
            pct = (count / len(grades)) * 100
            print(f"{grade}: {count} students ({pct:.1f}%)")
        
        output_file = input("Enter output filename (please enter 'results.txt'): ").strip()
        if not output_file:
            output_file = 'results.txt'
        
        with open(output_file, 'w') as f:
            f.write("Reg Num,Exam,Coursework,Overall,Grade\n")
            for student in sorted_array:
                f.write(f"{student['reg_num']},{student['exam']:.1f},{student['coursework']:.1f},"
                       f"{student['overall']:.2f},{student['grade']}\n")
        
        print("\n ALL STUDENTS (Sorted by Overall Mark):")
        for i, student in enumerate(sorted_array, 1):
            print(f"{i:2d}. {student['reg_num']:8s} | {student['exam']:5.1f} | {student['coursework']:5.1f} | {student['overall']:5.2f} | {student['grade']}")
        
            
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found!")
    except ValueError as e:
        print(f"Error parsing data: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    main()

Enter input filename(enter marks.txt):  marks.txt



 Grade Statistics:
A: 3 students (60.0%)
B: 1 students (20.0%)
C: 1 students (20.0%)


Enter output filename (please enter 'results.txt'):  results.txt



 ALL STUDENTS (Sorted by Overall Mark):
 1. STU003   |  95.0 |  82.0 | 91.10 | A
 2. STU002   |  92.0 |  88.0 | 90.80 | A
 3. STU001   |  85.0 |  78.0 | 82.90 | A
 4. STU004   |  63.0 |  70.0 | 65.10 | B
 5. STU005   |  55.0 |  55.0 | 55.00 | C


In [162]:
import requests
import csv
from datetime import datetime

# Fetch weather data from API
def fetch_weather(city: str, api_key: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()

        if response.status_code != 200:
            print(" API Error:", data.get("message", "Unknown error"))
            return {}

        return data

    except requests.exceptions.RequestException as e:
        print(" Network error:", e)
        return {}


# Analyze weather conditions
def analyze_weather(weather_data: dict) -> str:
    if not weather_data:
        return "No data to analyze."

    temp = weather_data["main"]["temp"]
    humidity = weather_data["main"]["humidity"]
    wind_speed = weather_data["wind"]["speed"]

    # Temperature conditions
    if temp <= 10:
        condition = "Cold (≤10°C)"
    elif 11 <= temp <= 24:
        condition = "Mild (11–24°C)"
    else:
        condition = "Hot (≥25°C)"

    alerts = []
    if wind_speed > 10:
        alerts.append("High wind alert!")
    if humidity > 80:
        alerts.append("Humid conditions!")

    if alerts:
        condition += " | " + " ".join(alerts)

    return condition


# Save weather data to CSV 
def save_to_csv(weather_data: dict, analysis: str, filename="weather_data.csv"):
    if not weather_data:
        print("No data to save.")
        return

    row = {
        "City": weather_data["name"],
        "Temperature (°C)": weather_data["main"]["temp"],
        "Humidity (%)": weather_data["main"]["humidity"],
        "Wind Speed (m/s)": weather_data["wind"]["speed"],
        "Weather Description": weather_data["weather"][0]["description"],
        "Analysis": analysis,
        "Date & Time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    try:
        with open(filename, "a", newline="", encoding="utf-8-sig") as file:
            writer = csv.DictWriter(file, fieldnames=row.keys())

            if file.tell() == 0:
                writer.writeheader()

            writer.writerow(row)

        print(f" Data saved to {filename}")

    except Exception as e:
        print(" CSV write error:", e)

# Main Program/ API_KEY function
if __name__ == "__main__":
    API_KEY = "65bef0f289c7946cfef78c98329fe368"
    city = input("Enter city name (city,country code): ").strip()

    weather_data = fetch_weather(city, API_KEY)
    analysis = analyze_weather(weather_data)

    print("\nWeather Analysis:")
    print(analysis)

    save_to_csv(weather_data, analysis)


Enter city name (city,country code):  Bengaluru



Weather Analysis:
Hot (≥25°C) | High wind alert!
✅ Data saved to weather_data.csv
